# Julia Draw Something

In [1]:
using DataFrames
using TensorFlow
using Distributions
using Images

I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:116] Couldn't open CUDA library libcudnn.so.5. LD_LIBRARY_PATH: /usr/local/cuda/lib64
I tensorflow/stream_executor/cuda/cuda_dnn.cc:3459] Unable to load cuDNN DSO
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcurand.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUD

In [2]:
data_set_folder = "/raided/datasets/sketches_png/png/"
list_of_imgs = open(data_set_folder * "filelist.txt")
img_paths = readlines(list_of_imgs)
close(list_of_imgs)

In [3]:
train_paths = Array(String, 700)
path_index = 1
count = 1
for i = 1:700
    train_paths[i] = img_paths[path_index]
    path_index += 1
    count += 1
    if count % 71 == 0
        count = 1
        path_index += 10
    end
end

test_paths = Array(String, 100)
path_index = 71
count = 1
for i = 1:100
    test_paths[i] = img_paths[path_index]
    path_index += 1
    count += 1
    if count % 11 == 0
        path_index += 70
        count = 1
    end
end

In [4]:
files_in_folder = readdir(data_set_folder)
labels = filter(x->isdir(data_set_folder * x) == true, files_in_folder)

250-element Array{String,1}:
 "airplane"     
 "alarm clock"  
 "angel"        
 "ant"          
 "apple"        
 "arm"          
 "armchair"     
 "ashtray"      
 "axe"          
 "backpack"     
 "banana"       
 "barn"         
 "baseball bat" 
 ⋮              
 "umbrella"     
 "van"          
 "vase"         
 "violin"       
 "walkie talkie"
 "wheel"        
 "wheelbarrow"  
 "windmill"     
 "wine-bottle"  
 "wineglass"    
 "wrist-watch"  
 "zebra"        

In [5]:
x = TensorFlow.placeholder(Float32)
y_ = TensorFlow.placeholder(Float32)
W = get_variable("weights", [77841, 10], Float32)
b = get_variable("bias", [10], Float32)

TensorFlow.Variable(<Operation 'bias'>,<Operation 'bias/Assign'>)

In [6]:
y = nn.softmax(x*W + b)
cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), reduction_indices=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


<Tensor NoOp:1 shape=unknown dtype=?>

In [7]:
type DataLoader
    cur_id::Int
    order::Vector{Int}
end

TrainLoader() = DataLoader(1, shuffle(1:700))
TestLoader() = DataLoader(1, shuffle(1:100))

TestLoader (generic function with 1 method)

In [8]:
function next_batch(loader::DataLoader, batch_size)
    x = zeros(Float32, batch_size, 77841)
    y = zeros(Float32, batch_size, 10)
    for i = 1:batch_size
        path = train_paths[loader.order[loader.cur_id]][1:end-1]
        println(path)
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 700
            break
        end
    end
    x, y
end

next_batch (generic function with 1 method)

In [9]:
function get_test(loader::DataLoader)
    x = zeros(Float32, 100, 77841)
    y = zeros(Float32, 100, 10)
    for i = 1:100
        path = test_paths[loader.order[loader.cur_id]][1:end-1]
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        println("--------------------------")
        println(label)
        println(loader.cur_id)
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 100
            println("========================")
            println(sketch_name)
            break
        end
    end
    x, y
end

get_test (generic function with 1 method)

In [10]:
loader = TrainLoader()
sess = Session()
run(sess, initialize_all_variables())
for i in 1:70
    batch = next_batch(loader, 10)
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
    println("DONE BATCH ", i)
end

armchair/519.png


I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1070
major: 6 minor: 1 memoryClockRate (GHz) 1.683
pciBusID 0000:01:00.0
Total memory: 7.92GiB
Free memory: 7.82GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0)


armchair/520.png
alarm clock/99.png
airplane/28.png
ashtray/603.png
axe/701.png
arm/423.png
arm/418.png
ant/282.png
angel/198.png
DONE BATCH 1
ashtray/582.png
angel/168.png
alarm clock/152.png
backpack/731.png
arm/441.png
ashtray/561.png
ashtray/599.png
ant/252.png
backpack/753.png
airplane/49.png
DONE BATCH 2
alarm clock/128.png
ant/320.png
backpack/775.png
apple/386.png
backpack/773.png
angel/230.png
ant/279.png
airplane/24.png
apple/381.png
arm/461.png
DONE BATCH 3
angel/167.png
alarm clock/148.png
arm/452.png
ant/272.png
backpack/768.png
apple/366.png
backpack/762.png
armchair/488.png
apple/331.png
angel/239.png
DONE BATCH 4
ashtray/584.png
ant/278.png
arm/449.png
ant/301.png
alarm clock/150.png
arm/438.png
ant/261.png
angel/178.png
angel/171.png
axe/679.png
DONE BATCH 5
backpack/727.png
apple/385.png
axe/707.png
apple/336.png
apple/351.png
ashtray/608.png
ant/275.png
backpack/799.png
ashtray/607.png
ashtray/589.png
DONE BATCH 6
backpack/780.png
apple/379.png
armchair/539.png
ant/2

LoadError: BoundsError: attempt to access 10×10 Array{Float32,2} at index [7,11]

In [11]:
correct_prediction = indmax(y, 2) .== indmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float64))
test_loader = TestLoader()
testx, testy = get_test(test_loader)

println(run(sess, indmax(y, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, indmax(y_, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

--------------------------
1
1
--------------------------
10
2
--------------------------
2
3
--------------------------
5
4
--------------------------
4
5
--------------------------
6
6
--------------------------
10
7
--------------------------
8
8
--------------------------
4
9
--------------------------
6
10
--------------------------
9
11
--------------------------
9
12
--------------------------
8
13
--------------------------
8
14
--------------------------
6
15
--------------------------
6
16
--------------------------
1
17
--------------------------
3
18
--------------------------
2
19
--------------------------
3
20
--------------------------
7
21
--------------------------
4
22
--------------------------
10
23
--------------------------
4
24
--------------------------
7
25
--------------------------
1
26
--------------------------
2
27
--------------------------
8
28
--------------------------
9
29
--------------------------
2
30
--------------------------
8
31
--------------

In [12]:
function sketch_input(path, sketch_name)
    x = zeros(Float32, 1, 77841)
    y = zeros(Float32, 1, 10)
    img = load(path)
    img = restrict(restrict(img))
    flatten_img = reshape(img, 77841)
    x[1, :] = flatten_img
    
    label = find((x -> x == sketch_name), labels)[1]
    y[1, label] = 1.0
    
    x, y
end

sketch_input (generic function with 1 method)

In [19]:
sketch_path = "/raided/datasets/sketches_png/png/axe/711.png"
sketch_label = "axe"

test_img, test_label = sketch_input(sketch_path, sketch_label)
what_it_thinks = labels[run(sess, indmax(y,2), Dict(x=>test_img, y_=>test_label))[1] + 1]
println("Is it an " * what_it_thinks * "? Answer: ", (sketch_label == what_it_thinks), ".")

Is it an axe? Answer: true.
